In [9]:
import os
from PIL import Image, ImageDraw, ImageFont

# Create output directory
output_dir = "synthetic_digits_cropped"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Parameters
image_size = (185, 386)  # Resolution of the image (width, height)
num_images_per_digit = 10  # Number of images per digit
digits = "0123456789"  # Digits to generate

# Function to calculate maximum font size to fill the image
def get_max_font_size(digit, image_size):
    test_font_size = 10
    while True:
        try:
            font = ImageFont.truetype("arial.ttf", test_font_size)
        except:
            font = ImageFont.load_default()
            break
        # Create temporary image to test text size
        temp_image = Image.new("RGB", image_size, "white")
        draw = ImageDraw.Draw(temp_image)
        bbox = draw.textbbox((0, 0), digit, font=font)
        text_width = bbox[2] - bbox[0]
        text_height = bbox[3] - bbox[1]
        if text_width > image_size[0] or text_height > image_size[1]:
            break
        test_font_size += 5
    return test_font_size - 5

# Function to generate digit image and crop it
def generate_digit_image(digit, output_path):
    # Create a blank image with white background
    image = Image.new("RGB", image_size, "white")
    draw = ImageDraw.Draw(image)

    # Dynamically calculate the maximum font size
    max_font_size = get_max_font_size(digit, image_size)
    font = ImageFont.truetype("arial.ttf", max_font_size) if "arial.ttf" else ImageFont.load_default()
    
    # Calculate text position to center the number
    bbox = draw.textbbox((0, 0), digit, font=font)
    text_width = bbox[2] - bbox[0]
    text_height = bbox[3] - bbox[1]
    
    # Shift text upward by reducing vertical position
    vertical_offset = 20  # Adjust this value to reduce white space (increase to move text up)
    text_position = ((image_size[0] - text_width) // 2, 
                     (image_size[1] - text_height) // 2 - vertical_offset)

    # Draw the digit
    draw.text(text_position, digit, fill="black", font=font)

    # Crop the image to remove white space
    non_white_bbox = image.getbbox()
    cropped_image = image.crop(non_white_bbox)
    
    # Save the cropped image
    cropped_image.save(output_path)


# Generate dataset
for digit in digits:
    digit_dir = os.path.join(output_dir, digit)
    os.makedirs(digit_dir, exist_ok=True)
    for i in range(num_images_per_digit):
        output_path = os.path.join(digit_dir, f"{digit}_{i}.png")
        generate_digit_image(digit, output_path)

print(f"Synthetic dataset created in '{output_dir}'")


Synthetic dataset created in 'synthetic_digits_cropped'
